# Imports

In [3]:
import os
import numpy as np

from tqdm import tqdm

import matplotlib.pyplot as plt

In [4]:
from Nim.Nim import Nim
from Nim.NimLogic import NimLogic

from Agents.Minimax.MinimaxAgentV1 import MinimaxAgentV1
from Agents.Minimax.MinimaxAgentV2 import MinimaxAgentV2
from Agents.Minimax.MinimaxAgentV3 import MinimaxAgentV3

# Constants

In [5]:
# Random seed for reproducibility
np.random.seed(42)

# Training parameters
EPISODES = 10000

# Game parameters
PILE_COUNT = 4
MAX_PILE = 127

# Agent Testing Function

In [6]:
def test_agent(_misereAgent, _normalAgent, _misere, _initial_piles):
    explored_nodes = np.ndarray(EPISODES)
    moves_count = np.ndarray(EPISODES)
    mean_nodes = np.ndarray(EPISODES)

    for i in tqdm(range(EPISODES)):
        game = Nim(
            initial_piles=_initial_piles[i],
            misere=_misere[i]
        )

        agent = _misereAgent if _misere[i] else _normalAgent

        winner = game.play(
            player1=agent,
            player2=agent,
            verbose=False
        )

        assert winner == NimLogic.is_p_position(_initial_piles[i], _misere[i])

        agent.compute_mean_nodes()

        mean_nodes[i] = agent.mean_nodes
        moves_count[i] = agent.moves_count
        explored_nodes[i] = agent.nodes_explored

    weighted_mean_nodes = explored_nodes.sum() / moves_count.sum()
    unweighted_mean_nodes = mean_nodes.mean()

    avg_explored_nodes = explored_nodes.mean()
    avg_moves_count = moves_count.mean()

    values = [unweighted_mean_nodes, weighted_mean_nodes, avg_explored_nodes, avg_moves_count]

    labels = [
        "unweighted average explored nodes per move:",
        "weighted average explored nodes per move:",
        "average explored nodes per game:",
        "average moves per game:"
    ]

    label_width = max(len(lbl) for lbl in labels)

    for lbl, val in zip(labels, values):
        print(f"{lbl:<{label_width}} {val:>10.2f}")

    return explored_nodes, moves_count, mean_nodes

# Random Game All Agent Test

In [ ]:
def run_tests(agent_v1, agent_v2, agent_v3, normal_v1, normal_v2, normal_v3, max_depth, pile_count, max_pile, episodes, save_path_prefix):
    initial_piles = np.random.randint(1, max_pile, size=(episodes, pile_count))
    misere_modes = np.random.choice([True, False], size=episodes)

    print("-" * 60)
    print(f"\tTestare cu max_depth={max_depth}, pile_count={pile_count}, max_pile={max_pile}")
    print("-" * 60)

    results = {
        'v1': test_agent(agent_v1, normal_v1, misere_modes, initial_piles),
        'v2': test_agent(agent_v2, normal_v2, misere_modes, initial_piles),
        'v3': test_agent(agent_v3, normal_v3, misere_modes, initial_piles),
    }

    for version, result in results.items():
        filename = f"{save_path_prefix}/minimax-{version}-{max_depth}-{pile_count}-{max_pile}.npz"
        np.savez(filename, result)

In [ ]:
os.makedirs("savedData/Minimax", exist_ok=True)

for max_depth in [1, 2]:
    misere_agents = [
        MinimaxAgentV1(misere=True, max_depth=max_depth),
        MinimaxAgentV2(misere=True, max_depth=max_depth),
        MinimaxAgentV3(misere=True, max_depth=max_depth)
    ]

    normal_agents = [
        MinimaxAgentV1(misere=False, max_depth=max_depth),
        MinimaxAgentV2(misere=False, max_depth=max_depth),
        MinimaxAgentV3(misere=False, max_depth=max_depth)
    ]

    for pile_count in [2, 4, 8]:
        run_tests(*misere_agents, *normal_agents, max_depth, pile_count, MAX_PILE, EPISODES, "savedData/Minimax")

    for max_pile in [63, 255]:
        run_tests(*misere_agents, *normal_agents, max_depth, PILE_COUNT, max_pile, EPISODES, "savedData/Minimax")

# Plotting results

In [63]:
def plot_results(vary_parameter, vary_values, fixed_pile_count, fixed_max_pile, folder="savedData/Minimax", save_folder="savedPlots/Minimax"):
    os.makedirs(save_folder, exist_ok=True)

    models = ['v1', 'v2', 'v3']
    model_names = ['Minimax V1', 'Minimax V2', 'Minimax V3']
    colors = ['#4B0082', '#4169E1', '#9370DB']
    metric_labels = [
        'Unweighted Avg Nodes/Move',
        'Weighted Avg Nodes/Move',
        'Avg Nodes/Game',
        'Avg Moves/Game'
    ]

    for max_depth in [1, 2]:
        fig, axes = plt.subplots(2, 2, figsize=(10, 7), facecolor='white', sharex=True)
        axes = axes.ravel()

        all_metrics = {model: {i: [] for i in range(4)} for model in models}

        for vary_value in vary_values:
            for model in models:
                pile_count = vary_value if vary_parameter == "pile_count" else fixed_pile_count
                max_pile = fixed_max_pile if vary_parameter == "pile_count" else vary_value

                filename = f"{folder}/minimax-{model}-{max_depth}-{pile_count}-{max_pile}.npz"
                if not os.path.exists(filename):
                    print(f"⚠️ Lipsă fișier: {filename}")
                    for i in range(4):
                        all_metrics[model][i].append(np.nan)
                    continue

                data = np.load(filename)
                test_data = data.get('arr_0')
                explored_nodes, moves_count, mean_nodes = test_data

                metrics_values = [
                    mean_nodes.mean(),
                    explored_nodes.sum() / moves_count.sum(),
                    explored_nodes.mean(),
                    moves_count.mean()
                ]

                for i in range(4):
                    all_metrics[model][i].append(metrics_values[i])

        for metric_idx, ax in enumerate(axes):
            for model_idx, model in enumerate(models):
                ax.plot(
                    vary_values,
                    all_metrics[model][metric_idx],
                    color=colors[model_idx],
                    marker='o',
                    alpha=0.5,
                    label=model_names[model_idx]
                )
            ax.set_ylabel(metric_labels[metric_idx], fontsize=16)
            ax.set_xlabel(vary_parameter.upper(), fontsize=16)
            ax.grid(True, alpha=0.3)
            if metric_idx == 0:
                ax.legend()

        fig.suptitle(f"Variation over {vary_parameter} with search depth of {max_depth}", fontsize=16)

        plt.tight_layout(rect=[0, 0, 1, 0.99])

        save_path = os.path.join(save_folder, f"variation_{vary_parameter}_depth_{max_depth}.png")
        plt.savefig(save_path, facecolor='white')
        plt.close()


In [64]:
plot_results(vary_parameter="pile_count", vary_values=[2, 4, 8], fixed_pile_count=4, fixed_max_pile=127)
plot_results(vary_parameter="max_pile", vary_values=[63, 127, 255], fixed_pile_count=4, fixed_max_pile=127)